# **DML**

In [0]:
data = [(5,100,'aa',1),(6,200,'bb',1),(7,300,'cc',1),(8,400,'dd',1)]

df = spark.createDataFrame(data, ['cust_id','income','name','tip'])

df.write.format("delta")\
        .mode("append")\
        .save("/Volumes/deltalakeansh/default/deltavol/dmlsink/")

In [0]:
%sql
SELECT * FROM delta.`/Volumes/deltalakeansh/default/deltavol/dmlsink/`

cust_id,income,name,tip
5,100,aa,1
6,200,bb,1
7,300,cc,1
8,400,dd,1
1,100,aa,1
2,200,bb,1
3,300,cc,1
4,400,dd,1


### **update**

In [0]:
%sql
UPDATE delta.`/Volumes/deltalakeansh/default/deltavol/dmlsink/`
SET income = 1000 WHERE cust_id = 5

num_affected_rows
1


In [0]:
%sql
SELECT * FROM delta.`/Volumes/deltalakeansh/default/deltavol/dmlsink/`

cust_id,income,name,tip
1,100,aa,1
2,200,bb,1
3,300,cc,1
4,400,dd,1
5,1000,aa,1
6,200,bb,1
7,300,cc,1
8,400,dd,1


# **UPSERT**

In [0]:
data = [(1,100,'xyz',1),(9,200,'bb',1),(10,300,'cc',1)]

df = spark.createDataFrame(data, ['cust_id','income','name','tip'])

display(df)


cust_id,income,name,tip
1,100,xyz,1
9,200,bb,1
10,300,cc,1


In [0]:
from delta.tables import DeltaTable

In [0]:
dlt_obj = DeltaTable.forPath(spark,"/Volumes/deltalakeansh/default/deltavol/dmlsink/")

dlt_obj.alias("trg").merge(df.alias("src"),"trg.cust_id = src.cust_id")\
            .whenMatchedUpdateAll()\
            .whenNotMatchedInsertAll()\
            .execute()

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
SELECT * FROM delta.`/Volumes/deltalakeansh/default/deltavol/dmlsink/`

cust_id,income,name,tip
2,200,bb,1
3,300,cc,1
4,400,dd,1
5,1000,aa,1
6,200,bb,1
7,300,cc,1
8,400,dd,1
1,100,xyz,1
9,200,bb,1
10,300,cc,1
